In [ ]:
# what model am i selecting
# why am i selecting the model
# evaluate the results
# use test data
# possibly try another model
# end with conclusion

# Introduction:
In this notebook, I will present my solution and analysis of the Titanic dataset.  This is a very famous dataset that can be found on [kaggle](https://www.kaggle.com/c/titanic/data). In this dataset we are given demographics of the Titanc passengers, incluiding who survived and who did ont. The goal is to build a model that can correctly classify who will survive using supervised machine learning. Throughout this notebook I will visualize the data, explain some data preprocessing techniques, construct and evaluate models and analyze the results. 

# Project Overview:
The primary objective of this project is to delve into the Titanic dataset, which contains information about passengers, including their demographics and survival status. By carefully studying the data, I will identify meaningful patterns and relationships that could help me in predicting survival outcomes. To achieve this, I will follow a structured approach: 

#### Data Exploration & Preprocessing:  
I will conduct a comprehensive exploration of the dataset, analyzing its various features, checking for missing values, and gaining insights into the distribution of variables. Prior to building the models, I will preprocess the data by handling missing values, encoding categorical variables, and scaling numerical features to ensure optimal model performance.

#### Model Building & Evaluation:
In this notebook I will try implement several models to try and correctly classify passengers who survied. The models I have chosen are logistic regression, decision trees, random forests, support vector machines and neural networks. For each of these modules I will evalute their peformance using f1score, confusion matrices, and overall accuracy. 

#### Conclusion: 
Finally, I will interpret the results of the models, identifying significant factors that contribute to passenger survival prediction and discussing potential areas for model improvement. The Titanic dataset is a good challange to test your knowledege on machine learning. This will serve as a good test for me to keep learning and testing my skills.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data Exploration
As mentioned earlier I got the dataset from kaggle. The link to that can be found above. The download came with two csv files. One for the training set and one for the test set. Since I have it locally on my computer I can eassily access the data as shown below. Some of the first steps we will do before creating a model is to see what our data looks like.

In [2]:
# read train and test sets
train = pd.read_csv('./train.csv') 
test = pd.read_csv('./test.csv') 

We loaded in the data into pandas dataframes and now we want to see what our data looks like. What does vairables does it contain and what data types, etc. First lets start by getting its size. 

In [8]:
len(train) # get size of train data

891

As we can see it is somewhat large. This will be good for our model to have many samples to learn from. Additionally its not too large where we would require lots of compute power and time for training. Moving on from this we can check what our data consists of. 

In [10]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [11]:
train.info() # get info on our train 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


The tables above tell us a couple of things. For starters we have see that we have 12 columns which are PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked. Lets breakdown each of them and see what might be useful for training our model. 

PassengderId: This column might not be vary useful as it is simply a id assigned by the dataset. We will drop this column before training. 

Survived: This column is our labels and is important since this is a supervised machine learning problem. If we wanted to go with a clustering/unsupervised learning we could drop this. For the purpose of the notebook we will be keeping this. 

Pclass: On kaggle it says that this columns serves "A proxy for socio-economic status". Where 1 is upper, 2 is middle and 3 is upper. This will be important for our model

Name: This is simply the name of the passenger. This could be important as some family last names could mean that they are from a wealthier family and therfore might have a higher chance of surviving. For this notebook we will most likely drop it. 

Sex/Age: The sex of the passenger. Since women and children were  prioritized in the case of an emergency this would helpful to determined who would survive. 



In [ ]:
# drop the columns PassengerId, Name, Ticket, Cabin
train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


Our dataframe now only contains the columns that we want. However from the table above we can see that we still have quite a few missing values in a couple of columns. For example age has quite a few missing values. Embarked has some missing values but nothing major. We now have to figure out what to do with the missing values for age. Lets first check how many missing values we have exactly. 

In [17]:
# get missing values 
train.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

We have a total of 177 missing values. We have a couple of options that we can do. We can either drop the rows of the missing values, in the case of embarked we probably will as it is only 2. For the case of age however we might want to try to fill the missing values. After researching on approaches I have found the method of ...... However I will be taking two approaches, one where we drop the rows that have missing values and one where we fill in the missing values using "". At the end we will see if there is really a difference and if it was worth filling in the missing values. 

In [19]:
# get the rows that have missing values
missing_vals = train[train.isna().any(axis=1)]

In [28]:
# show the first 15 samples in the dataframe of missing values
missing_vals.head(15) 

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
5,0,3,male,NaN,0,0,8.4583,Q
17,1,2,male,NaN,0,0,13.0000,S
19,1,3,female,NaN,0,0,7.2250,C
26,0,3,male,NaN,0,0,7.2250,C
28,1,3,female,NaN,0,0,7.8792,Q
29,0,3,male,NaN,0,0,7.8958,S
31,1,1,female,NaN,1,0,146.5208,C
32,1,3,female,NaN,0,0,7.7500,Q
36,1,3,male,NaN,0,0,7.2292,C
42,0,3,male,NaN,0,0,7.8958,C


From the table above we can see that from the first 15 samples most of the Pclass are 3. Lets count how many of the missing values are of Pclass 3. 

In [33]:
# in missing values count the pclass frequencies
missing_vals['Pclass'].value_counts()

Pclass
3    136
1     32
2     11
Name: count, dtype: int64

From this we can see that almost all missing values are from the Pclass 3. The reason for this could be